In [37]:
# auto reload notebook deps
%reload_ext autoreload
%autoreload 2

In [38]:
import json
import pandas as pd

DATASETS = {
    "CentralParkNYC": {
        "input_location": "./data/tweet_objects/CentralParkNYC/2021-02-01-2021-02-03.json",
        "output_location": "./data/mstream_datasets/CentralParkNYC-2021-02-01-2021-02-03.json",
        "spike_start": "Feb 1, 2021, 17:00",
        "spike_end": "Feb 3, 2021, 04:00",
    }
}

selected_dataset = DATASETS["CentralParkNYC"]
labeled_spike_start = pd.to_datetime(selected_dataset["spike_start"]).tz_localize("UTC")
labeled_spike_end = pd.to_datetime(selected_dataset["spike_end"]).tz_localize("UTC")

In [39]:
from utils.dataset import create_tweet_df

with open(selected_dataset["input_location"], "r") as f:
    raw_dataset = json.load(f)

df_tweets = create_tweet_df(raw_dataset["tweets"])

# Label tweets
df_tweets["is_anomaly"] = df_tweets.created_at.apply(
    lambda x: 0 if (x < labeled_spike_start or x > labeled_spike_end) else 1
)

df_tweets.head()

,id,text,created_at,hashtags,mentions,in_reply_to_user_id,replied_to,retweeted,quoted,is_anomaly
1999,1356033603138949130,RT @ivanguerramex: @GustavoGarciaMH @Espejored...,2021-02-01 00:16:19+00:00,[],"[ivanguerramex, GustavoGarciaMH, Espejored, qu...",None,NaN,1355577292974657542,NaN,0
1998,1356036627487985665,"RT @leostamillo: No, we didn’t have the chance...",2021-02-01 00:28:20+00:00,[snowyowl],"[leostamillo, CentralParkNYC]",None,NaN,1354970865201537026,NaN,0
1997,1356047582196342785,RT @ivanguerramex: @GustavoGarciaMH @Espejored...,2021-02-01 01:11:52+00:00,[],"[ivanguerramex, GustavoGarciaMH, Espejored, qu...",None,NaN,1292121216518623236,NaN,0
1996,1356060552628494336,@prospect_park and @CentralParkNYC could do th...,2021-02-01 02:03:24+00:00,[],"[prospect_park, CentralParkNYC, RiversideParkNY]",18725659,NaN,NaN,NaN,0
1995,1356070453572489234,RT @ZubizuBags: This #wingsair tourist helicop...,2021-02-01 02:42:45+00:00,[wingsair],"[ZubizuBags, CentralParkNYC, FAANews]",None,NaN,1355601330010812416,NaN,0


In [40]:
# Export 
df_tweets.to_json(
    selected_dataset["output_location"],
    orient="records"
)